<a href="https://colab.research.google.com/github/mhbw/Discogs-Postman/blob/master/Copy_of_Julia_Colab_Notebook_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template: MIT ATTEMPT_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = 1 ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.1 on the current Colab Runtime...
2022-09-22 00:39:16 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.1-linux-x86_64.tar.gz [137332197/137332197] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:

versioninfo()

Julia Version 1.8.1
Commit afb6c60d69a (2022-09-06 15:09 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  LD_PRELOAD = /usr/lib/x86_64-linux-gnu/libtcmalloc.so.4
  JULIA_NUM_THREADS = 2


In [ ]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  445.380 ms (2 allocations: 32.00 MiB)


In [ ]:
if ENV["COLAB_GPU"] == 1
    using CUDA

    run(`nvidia-smi`)

    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
else
    println("No GPU found.")
end

Wed Sep 21 16:02:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

From here this is following Dr Hunter's project, [Fantasy Hockey IP code](https://github.com/dscotthunter/Fantasy-Hockey-IP-Code).

In [3]:
using Pkg
Pkg.add("JuMP")
Pkg.add("DataFrames")
Pkg.add("GLPKMathProgInterface")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed CodecBzip2 ─────────── v0.7.2
   Installed StaticArraysCore ───── v1.3.0
   Installed CommonSubexpressions ─ v0.3.0
   Installed DiffRules ──────────── v1.11.1
   Installed MutableArithmetics ─── v1.0.4
   Installed StaticArrays ───────── v1.5.7
   Installed DiffResults ────────── v1.0.3
   Installed JuMP ───────────────── v1.3.0
   Installed ForwardDiff ────────── v0.10.32
   Installed MathOptInterface ───── v1.8.2
    Updating `~/.julia/environments/v1.8/Project.toml`
  [4076af6c] + JuMP v1.3.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [523fee87] + CodecBzip2 v0.7.2
  [bbf7d656] + CommonSubexpressions v0.3.0
  [163ba53b] + DiffResults v1.0.3
  [b552c78f] + DiffRules v1.11.1
  [f6369f11] + ForwardDiff v0.10.32
  [4076af6c] + JuMP v1.3.0
  [b8f27783] + MathOptInterface v1.8.2
  [d8a4904e] + MutableArithmetics v1.0.4
  [90137ffa] + StaticArrays v1.5.7
⌅ [1e83bf80] + 

Import the data from the github

In [4]:
 Pkg.add("Git")
using Git
run(`$(git()) clone https://github.com/dscotthunter/Fantasy-Hockey-IP-Code`)


   Resolving package versions...
   Installed Git_jll ─ v2.34.1+0
   Installed Git ───── v1.2.1
    Updating `~/.julia/environments/v1.8/Project.toml`
  [d7ba0133] + Git v1.2.1
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [d7ba0133] + Git v1.2.1
  [f8c6e375] + Git_jll v2.34.1+0
  [efcefdf7] + PCRE2_jll v10.40.0+0
Precompiling project...
  ✓ PCRE2_jll
  ✓ Git_jll
  ✓ Git
  3 dependencies successfully precompiled in 1 seconds. 162 already precompiled.
Cloning into 'Fantasy-Hockey-IP-Code'...


Process(setenv(`/root/.julia/artifacts/33c5e3a13ad6427f86436f577c0edce1e468ac80/bin/git clone https://github.com/dscotthunter/Fantasy-Hockey-IP-Code`,["ENV=/root/.bashrc", "PATH=/root/.julia/artifacts/fac7e6d8fc4c5775bf5118ab494120d2a0db4d64/bin:/root/.julia/artifacts/25fa81dbac6496585a91dbdc258273d39442466f/bin:/root/.julia/artifacts/3e19866657986912870f596aecfee137473965a9/bin:/root/.julia/artifacts/33c5e3a13ad6427f86436f577c0edce1e468ac80/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin", "NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.8.4-1+cuda11.1", "NV_LIBNCCL_PACKAGE_VERSION=2.8.4-1", "NV_LIBCUBLAS_PACKAGE_NAME=libcublas-11-1", "NV_NVPROF_VERSION=11.1.105-1", "NVIDIA_DRIVER_CAPABILITIES=compute,utility", "LD_LIBRARY_PATH=/usr/local/bin/../lib/julia:/root/.julia/artifacts/fac7e6d8fc4c5775bf5118ab494120d2a0db4d64/lib:/root/.julia/artifacts/25fa81dbac6496585a91dbdc258273d39442466f/

This is clunky but I'm going to try and import it raw: (edit: this failed massively, because, shocker, the code was very dated. many updates now). 

In [5]:
Pkg.add("Printf")
Pkg.add("CSV")
Pkg.add("HTTP")

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [de0858da] + Printf
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
   Installed InlineStrings ── v1.1.4
   Installed SentinelArrays ─ v1.3.14
   Installed WeakRefStrings ─ v1.4.2
   Installed FilePathsBase ── v0.9.20
   Installed CSV ──────────── v0.10.4
    Updating `~/.julia/environments/v1.8/Project.toml`
  [336ed68f] + CSV v0.10.4
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [336ed68f] + CSV v0.10.4
  [48062228] + FilePathsBase v0.9.20
  [842dd82b] + InlineStrings v1.1.4
  [91c51154] + SentinelArrays v1.3.14
  [ea10d353] + WeakRefStrings v1.4.2
Precompiling project...
  ✓ InlineStrings
  ✓ SentinelArrays
  ✓ FilePathsBase
  ✓ WeakRefStrings
  ✓ CSV
  5 dependencies successfully precompiled in 8 seconds. 165 already precompiled.
   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
⌅ [cd3eb016] + HTTP v0

In [6]:
using LinearAlgebra
Pkg.add("GLPK")

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
⌅ [60bf3e95] + GLPK v0.13.0
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

In [20]:
#=
This code implements the No Stacking, Type 1, Type 2, Type 3, Type 4, and Type 5 formulations
described in the paper Winning Daily Fantasy Hockey Contests Using Integer Programming by
Hunter, Vielma, and Zaman. We have made an attempt to describe the code in great detail, with the
hope that you will use your expertise to build better formulations.
=#

# To install DataFrames, simply run Pkg.add("DataFrames")
using DataFrames
using Printf
using CSV
using HTTP
#=
GLPK is an open-source solver, and additionally Cbc is an open-source solver. This code uses GLPK
because we found that it was slightly faster than Cbc in practice. For those that want to build
very sophisticated models, they can buy Gurobi. To install GLPKMathProgInterface, simply run
Pkg.add("GLPKMathProgInterface")
=#
using GLPKMathProgInterface
using GLPK
# Once again, to install run Pkg.add("JuMP")
using JuMP

#=
Variables for solving the problem (change these)
=#
# num_lineups is the total number of lineups
num_lineups = 1

# num_overlap is the maximum overlap of players between the lineups that you create
num_overlap = 3

# path_skaters is a string that gives the path to the csv file with the skaters information (see example file for suggested format)
path_skaters = "https://raw.githubusercontent.com/dscotthunter/Fantasy-Hockey-IP-Code/master/example_skaters.csv"

# path_goalies is a string that gives the path to the csv file with the goalies information (see example file for suggested format)
path_goalies = "https://raw.githubusercontent.com/dscotthunter/Fantasy-Hockey-IP-Code/673b0a5119ed746a8dc2347206d138c84407def1/example_goalies.csv"

# path_to_output is a string that gives the path to the csv file that will give the outputted results
path_to_output= "output.csv"



# This is a function that creates one lineup using the No Stacking formulation from the paper
function one_lineup_no_stacking(skaters, goalies, lineups, num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    m = Model(with_optimizer(GLPK.Optimizer))

    # Variable for skaters in lineup.
    @variable(m, skaters_lineup[i=1:num_skaters], Bin)

    # Variable for goalie in lineup.
    @variable(m, goalies_lineup[i=1:num_goalies], Bin)


    # One goalie constraint
    @constraint(m, sum(goalies_lineup[i], i=1:num_goalies) == 1)

    # Eight Skaters constraint
    @constraint(m, sum(skaters_lineup[i], i=1:num_skaters) == 8)

    # between 2 and 3 centers
    @constraint(m, sum(centers[i]*skaters_lineup[i], i=1:num_skaters) <= 3)
    @constraint(m, 2 <= sum(centers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 3 and 4 wingers
    @constraint(m, sum(wingers[i]*skaters_lineup[i], i=1:num_skaters) <= 4)
    @constraint(m, 3<=sum(wingers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 2 and 3 defenders
    @constraint(m, 2 <= sum(defenders[i]*skaters_lineup[i], i=1:num_skaters))
    @constraint(m, sum(defenders[i]*skaters_lineup[i], i=1:num_skaters) <= 3)

    # Financial Constraint
    @constraint(m, sum(skaters[i,:Salary]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Salary]*goalies_lineup[i], i=1:num_goalies) <= 50000)

    # at least 3 different teams for the 8 skaters constraints
    @variable(m, used_team[i=1:num_teams], Bin)
    @constraint(m, constr[i=1:num_teams], used_team[i] <= sum(skaters_teams[t, i]*skaters_lineup[t], t=1:num_skaters))
    @constraint(m, sum(used_team[i], i=1:num_teams) >= 3)

    # Overlap Constraint
    @constraint(m, constr[i=1:size(lineups)[2]], sum(lineups[j,i]*skaters_lineup[j], j=1:num_skaters) + sum(lineups[num_skaters+j,i]*goalies_lineup[j], j=1:num_goalies) <= num_overlap)


    # Objective
    @objective(m, Max, sum(skaters[i,:Projection]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Projection]*goalies_lineup[i], i=1:num_goalies))


    # Solve the integer programming problem
    println("Solving Problem...")
    @printf("\n")
    status = solve(m);


    # Puts the output of one lineup into a format that will be used later
    if status==:Optimal
        skaters_lineup_copy = Array{Int}(undef, 0)
        for i=1:num_skaters
            if getValue(skaters_lineup[i]) >= 0.9 && getValue(skaters_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        for i=1:num_goalies
            if getValue(goalies_lineup[i]) >= 0.9 && getValue(goalies_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        return(skaters_lineup_copy)
    end
end





# This is a function that creates one lineup using the Type 1 formulation from the paper
function one_lineup_Type_1(skaters, goalies, lineups, num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    m = Model(with_optimizer(GLPK.Optimizer))

    # Variable for skaters in lineup
    @variable(m, skaters_lineup[i=1:num_skaters], Bin)

    # Variable for goalie in lineup
    @variable(m, goalies_lineup[i=1:num_goalies], Bin)


    # One goalie constraint
    @constraint(m, sum(goalies_lineup[i], i=1:num_goalies) == 1)

    # Eight skaters constraint
    @constraint(m, sum(skaters_lineup[i], i=1:num_skaters) == 8)


    # between 2 and 3 centers
    @constraint(m, sum(centers[i]*skaters_lineup[i], i=1:num_skaters) <= 3)
    @constraint(m, 2 <= sum(centers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 3 and 4 wingers
    @constraint(m, sum(wingers[i]*skaters_lineup[i], i=1:num_skaters) <= 4)
    @constraint(m, 3<=sum(wingers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 2 and 3 defenders
    @constraint(m, 2 <= sum(defenders[i]*skaters_lineup[i], i=1:num_skaters))
    @constraint(m, sum(defenders[i]*skaters_lineup[i], i=1:num_skaters) <= 3)


    # Financial Constraint
    @constraint(m, sum(skaters[i,:Salary]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Salary]*goalies_lineup[i], i=1:num_goalies) <= 50000)


    # At least 3 different teams for the 8 skaters constraint
    @variable(m, used_team[i=1:num_teams], Bin)
    @constraint(m, constr[i=1:num_teams], used_team[i] <= sum(skaters_teams[t, i]*skaters_lineup[t], t=1:num_skaters))
    @constraint(m, sum(used_team[i], i=1:num_teams) >= 3)


    # No goalies going against skaters constraint
    @constraint(m, constr[i=1:num_goalies], 6*goalies_lineup[i] + sum(goalie_opponents[k, i]*skaters_lineup[k], k=1:num_skaters)<=6)


    # Must have at least one complete line in each lineup
    @variable(m, line_stack[i=1:num_lines], Bin)
    @constraint(m, constr[i=1:num_lines], 3*line_stack[i] <= sum(team_lines[k,i]*skaters_lineup[k], k=1:num_skaters))
    @constraint(m, sum(line_stack[i], i=1:num_lines) >= 1)


    # Must have at least 2 lines with at least two people
    @variable(m, line_stack2[i=1:num_lines], Bin)
    @constraint(m, constr[i=1:num_lines], 2*line_stack2[i] <= sum(team_lines[k,i]*skaters_lineup[k], k=1:num_skaters))
    @constraint(m, sum(line_stack2[i], i=1:num_lines) >= 2)


    # Overlap Constraint
    @constraint(m, constr[i=1:size(lineups)[2]], sum(lineups[j,i]*skaters_lineup[j], j=1:num_skaters) + sum(lineups[num_skaters+j,i]*goalies_lineup[j], j=1:num_goalies) <= num_overlap)


    # Objective
    @objective(m, Max, sum(skaters[i,:Projection]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Projection]*goalies_lineup[i], i=1:num_goalies) )


    # Solve the integer programming problem
    println("Solving Problem...")
    @printf("\n")
    status = solve(m);


    # Puts the output of one lineup into a format that will be used later
    if status==:Optimal
        skaters_lineup_copy = Array{Int}(undef, 0)
        for i=1:num_skaters
            if getValue(skaters_lineup[i]) >= 0.9 && getValue(skaters_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        for i=1:num_goalies
            if getValue(goalies_lineup[i]) >= 0.9 && getValue(goalies_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        return(skaters_lineup_copy)
    end
end





# This is a function that creates one lineup using the Type 2 formulation from the paper
function one_lineup_Type_2(skaters, goalies, lineups, num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    m = Model(with_optimizer(GLPK.Optimizer))

    # Variable for skaters in lineup
    @variable(m, skaters_lineup[i=1:num_skaters], Bin)

    # Variable for goalie in lineup
    @variable(m, goalies_lineup[i=1:num_goalies], Bin)


    # One goalie constraint
    @constraint(m, sum(goalies_lineup[i], i=1:num_goalies) == 1)

    # Eight skaters constraint
    @constraint(m, sum(skaters_lineup[i], i=1:num_skaters) == 8)


    # between 2 and 3 centers
    @constraint(m, sum(centers[i]*skaters_lineup[i], i=1:num_skaters) <= 3)
    @constraint(m, 2 <= sum(centers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 3 and 4 wingers
    @constraint(m, sum(wingers[i]*skaters_lineup[i], i=1:num_skaters) <= 4)
    @constraint(m, 3<=sum(wingers[i]*skaters_lineup[i], i=1:num_skaters))

    # exactly 2 defenders
    @constraint(m, 2 == sum(defenders[i]*skaters_lineup[i], i=1:num_skaters))

    # Financial Constraint
    @constraint(m, sum(skaters[i,:Salary]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Salary]*goalies_lineup[i], i=1:num_goalies) <= 50000)


    # at least 3 different teams for the 8 skaters constraint
    @variable(m, used_team[i=1:num_teams], Bin)
    @constraint(m, constr[i=1:num_teams], used_team[i] <= sum(skaters_teams[t, i]*skaters_lineup[t], t=1:num_skaters))
    @constraint(m, sum(used_team[i], i=1:num_teams) >= 3)


    # No goalies going against skaters constraint
    @constraint(m, constr[i=1:num_goalies], 6*goalies_lineup[i] + sum(goalie_opponents[k, i]*skaters_lineup[k], k=1:num_skaters)<=6)


    # Must have at least one complete line in each lineup
    @variable(m, line_stack[i=1:num_lines], Bin)
    @constraint(m, constr[i=1:num_lines], 3*line_stack[i] <= sum(team_lines[k,i]*skaters_lineup[k], k=1:num_skaters))
    @constraint(m, sum(line_stack[i], i=1:num_lines) >= 1)


    # Must have at least 2 lines with at least two people
    @variable(m, line_stack2[i=1:num_lines], Bin)
    @constraint(m, constr[i=1:num_lines], 2*line_stack2[i] <= sum(team_lines[k,i]*skaters_lineup[k], k=1:num_skaters))
    @constraint(m, sum(line_stack2[i], i=1:num_lines) >= 2)


    # Overlap Constraint
    @constraint(m, constr[i=1:size(lineups)[2]], sum(lineups[j,i]*skaters_lineup[j], j=1:num_skaters) + sum(lineups[num_skaters+j,i]*goalies_lineup[j], j=1:num_goalies) <= num_overlap)


    # Objective
    @objective(m, Max, sum(skaters[i,:Projection]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Projection]*goalies_lineup[i], i=1:num_goalies) )


    # Solve the integer programming problem
    println("Solving Problem...")
    @printf("\n")
    status = solve(m);


    # Puts the output of one lineup into a format that will be used later
    if status==:Optimal
        skaters_lineup_copy = Array{Int}(undef, 0)
        for i=1:num_skaters
            if getValue(skaters_lineup[i]) >= 0.9 && getValue(skaters_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        for i=1:num_goalies
            if getValue(goalies_lineup[i]) >= 0.9 && getValue(goalies_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        return(skaters_lineup_copy)
    end
end




# This is a function that creates one lineup using the Type 3 formulation from the paper
function one_lineup_Type_3(skaters, goalies, lineups, num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    m = Model(with_optimizer(GLPK.Optimizer))


    # Variable for skaters in lineup
    @variable(m, skaters_lineup[i=1:num_skaters], Bin)

    # Variable for goalie in lineup
    @variable(m, goalies_lineup[i=1:num_goalies], Bin)


    # One goalie constraint
    @constraint(m, sum(goalies_lineup[i], i=1:num_goalies) == 1)

    # Eight Skaters constraint
    @constraint(m, sum(skaters_lineup[i], i=1:num_skaters) == 8)


    # between 2 and 3 centers
    @constraint(m, sum(centers[i]*skaters_lineup[i], i=1:num_skaters) <= 3)
    @constraint(m, 2 <= sum(centers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 3 and 4 wingers
    @constraint(m, sum(wingers[i]*skaters_lineup[i], i=1:num_skaters) <= 4)
    @constraint(m, 3<=sum(wingers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 2 and 3 defenders
    @constraint(m, 2 <= sum(defenders[i]*skaters_lineup[i], i=1:num_skaters))
    @constraint(m, sum(defenders[i]*skaters_lineup[i], i=1:num_skaters) <= 3)


    # Financial Constraint
    @constraint(m, sum(skaters[i,:Salary]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Salary]*goalies_lineup[i], i=1:num_goalies) <= 50000)


    # at least 3 different teams for the 8 skaters constraint
    @variable(m, used_team[i=1:num_teams], Bin)
    @constraint(m, constr[i=1:num_teams], used_team[i] <= sum(skaters_teams[t, i]*skaters_lineup[t], t=1:num_skaters))
    @constraint(m, sum(used_team[i], i=1:num_teams) >= 3)



    # No goalies going against skaters
    @constraint(m, constr[i=1:num_goalies], 6*goalies_lineup[i] + sum(goalie_opponents[k, i]*skaters_lineup[k], k=1:num_skaters)<=6)

    # Must have at least one complete line in each lineup
    @variable(m, line_stack[i=1:num_lines], Bin)
    @constraint(m, constr[i=1:num_lines], 3*line_stack[i] <= sum(team_lines[k,i]*skaters_lineup[k], k=1:num_skaters))
    @constraint(m, sum(line_stack[i], i=1:num_lines) >= 1)


    # Must have at least 2 lines with at least two people
    @variable(m, line_stack2[i=1:num_lines], Bin)
    @constraint(m, constr[i=1:num_lines], 2*line_stack2[i] <= sum(team_lines[k,i]*skaters_lineup[k], k=1:num_skaters))
    @constraint(m, sum(line_stack2[i], i=1:num_lines) >= 2)



    # The defenders must be on Power Play 1 constraint
    @constraint(m, sum(sum(defenders[i]*P1_info[i,j]*skaters_lineup[i], i=1:num_skaters), j=1:num_teams) ==  sum(defenders[i]*skaters_lineup[i], i=1:num_skaters))


    # Overlap Constraint
    @constraint(m, constr[i=1:size(lineups)[2]], sum(lineups[j,i]*skaters_lineup[j], j=1:num_skaters) + sum(lineups[num_skaters+j,i]*goalies_lineup[j], j=1:num_goalies) <= num_overlap)



    # Objective
    @objective(m, Max, sum(skaters[i,:Projection]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Projection]*goalies_lineup[i], i=1:num_goalies) )


    # Solve the integer programming problem
    println("Solving Problem...")
    @printf("\n")
    status = solve(m);


    # Puts the output of one lineup into a format that will be used later
    if status==:Optimal
        skaters_lineup_copy = Array{Int}(undef, 0)
        for i=1:num_skaters
            if getValue(skaters_lineup[i]) >= 0.9 && getValue(skaters_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        for i=1:num_goalies
            if getValue(goalies_lineup[i]) >= 0.9 && getValue(goalies_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        return(skaters_lineup_copy)
    end
end




# This is a function that creates one lineup using the Type 4 formulation from the paper
function one_lineup_Type_4(skaters, goalies, lineups, num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    m = Model(with_optimizer(GLPK.Optimizer))


    # Variable for skaters in lineup
    @variable(m, skaters_lineup[i=1:num_skaters], Bin)

    # Variable for goalie in lineup
    @variable(m, goalies_lineup[i=1:num_goalies], Bin)


    # One goalie constraint
    @constraint(m, sum(goalies_lineup[i] for i=1:num_goalies)  == 1)

    # Eight Skaters constraint
    @constraint(m, sum(skaters_lineup[i] for i=1:num_skaters) == 8)

    # between 2 and 3 centers
    @constraint(m, sum(centers[i]*skaters_lineup[i] for i=1:num_skaters) <= 3)
    @constraint(m, 2 <= sum(centers[i]*skaters_lineup[i] for i=1:num_skaters))

    # between 3 and 4 wingers
    @constraint(m, sum(wingers[i]*skaters_lineup[i] for i=1:num_skaters) <= 4)
    @constraint(m, 3<=sum(wingers[i]*skaters_lineup[i] for i=1:num_skaters))

    # between 2 and 3 defenders
    @constraint(m, 2 <= sum(defenders[i]*skaters_lineup[i] for i=1:num_skaters))
    @constraint(m, sum(defenders[i]*skaters_lineup[i] for i=1:num_skaters) <= 3)


    # Financial Constraint
    @constraint(m, sum(skaters[i,:Salary]*skaters_lineup[i] for i=1:num_skaters) + sum(goalies[i,:Salary]*goalies_lineup[i] for i=1:num_goalies) <= 50000)


    # exactly 3 different teams for the 8 skaters constraint
    @variable(m, used_team[i=1:num_teams], Bin)
    @constraint(m, constr[i=1:num_teams], used_team[i] <= sum(skaters_teams[t, i]*skaters_lineup[t] for t=1:num_skaters))
    @constraint(m, constr2[i=1:num_teams], sum(skaters_teams[t, i]*skaters_lineup[t] for t=1:num_skaters) <= 6*used_team[i])
    @constraint(m, sum(used_team[i] for i=1:num_teams) == 3)


    # No goalies going against skaters
    @constraint(m, constr3[i=1:num_goalies], 6*goalies_lineup[i] + sum(goalie_opponents[k, i]*skaters_lineup[k] for k=1:num_skaters)<=6)


    # Must have at least one complete line in each lineup
    @variable(m, line_stack[i=1:num_lines], Bin)
    @constraint(m, constr4[i=1:num_lines], 3*line_stack[i] <= sum(team_lines[k,i]*skaters_lineup[k] for k=1:num_skaters))
    @constraint(m, sum(line_stack[i] for i=1:num_lines) >= 1)


    # Must have at least 2 lines with at least two people
    @variable(m, line_stack2[i=1:num_lines], Bin)
    @constraint(m, constr5[i=1:num_lines], 2*line_stack2[i] <= sum(team_lines[k,i]*skaters_lineup[k] for k=1:num_skaters))
    @constraint(m, sum(line_stack2[i] for i=1:num_lines) >= 2)



    # The defenders must be on Power Play 1
    @constraint(m, sum(sum(defenders[i]*P1_info[i,j]*skaters_lineup[i] for i=1:num_skaters) for j=1:num_teams) ==  sum(defenders[i]*skaters_lineup[i] for i=1:num_skaters))


    # Overlap Constraint
    @constraint(m, constr6[i=1:size(lineups)[2]], sum(lineups[j,i]*skaters_lineup[j] for j=1:num_skaters) + sum(lineups[num_skaters+j,i]*goalies_lineup[j] for  j=1:num_goalies) <= num_overlap)



    # Objective
    @objective(m, Max, sum(skaters[i,:Projection]*skaters_lineup[i] for i=1:num_skaters) + sum(goalies[i,:Projection]*goalies_lineup[i] for i=1:num_goalies) )


    # Solve the integer programming problem
    println("Solving Problem...")
    @printf("\n")
    status = optimize!(m);


    # Puts the output of one lineup into a format that will be used later
    if status==:Optimal
        skaters_lineup_copy = Array(Int, 0)
        for i=1:num_skaters
            if getValue(skaters_lineup[i]) >= 0.9 && getValue(skaters_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        for i=1:num_goalies
            if getValue(goalies_lineup[i]) >= 0.9 && getValue(goalies_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        return(skaters_lineup_copy)
    end
end


# This is a function that creates one lineup using the Type 5 formulation from the paper
function one_lineup_Type_5(skaters, goalies, lineups, num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    m = Model(with_optimizer(GLPK.Optimizer))

    # Variable for skaters in lineup
    @variable(m, skaters_lineup[i=1:num_skaters], Bin)

    # Variable for goalie in lineup
    @variable(m, goalies_lineup[i=1:num_goalies], Bin)


    # One goalie constraint
    @constraint(m, sum(goalies_lineup[i], i=1:num_goalies) == 1)

    # Eight skaters constraint
    @constraint(m, sum(skaters_lineup[i], i=1:num_skaters) == 8)



    # between 2 and 3 centers
    @constraint(m, sum(centers[i]*skaters_lineup[i], i=1:num_skaters) <= 3)
    @constraint(m, 2 <= sum(centers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 3 and 4 wingers
    @constraint(m, sum(wingers[i]*skaters_lineup[i], i=1:num_skaters) <= 4)
    @constraint(m, 3<=sum(wingers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 2 and 3 defenders
    @constraint(m, 2 <= sum(defenders[i]*skaters_lineup[i], i=1:num_skaters))
    @constraint(m, sum(defenders[i]*skaters_lineup[i], i=1:num_skaters) <= 3)


    # Financial Constraint
    @constraint(m, sum(skaters[i,:Salary]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Salary]*goalies_lineup[i], i=1:num_goalies) <= 50000)


    # exactly 3 different teams for the 8 skaters constraint
    @variable(m, used_team[i=1:num_teams], Bin)
    @constraint(m, constr[i=1:num_teams], used_team[i] <= sum(skaters_teams[t, i]*skaters_lineup[t], t=1:num_skaters))
    @constraint(m, constr[i=1:num_teams], sum(skaters_teams[t, i]*skaters_lineup[t], t=1:num_skaters) <= 6*used_team[i])
    @constraint(m, sum(used_team[i], i=1:num_teams) == 3)



    # No goalies going against skaters
    @constraint(m, constr[i=1:num_goalies], 6*goalies_lineup[i] + sum(goalie_opponents[k, i]*skaters_lineup[k], k=1:num_skaters)<=6)



    # Must have at least one complete line in each lineup
    @variable(m, line_stack[i=1:num_lines], Bin)
    @constraint(m, constr[i=1:num_lines], 3*line_stack[i] <= sum(team_lines[k,i]*skaters_lineup[k], k=1:num_skaters))
    @constraint(m, sum(line_stack[i], i=1:num_lines) >= 1)

    # Must have at least 2 lines with at least two people
    @variable(m, line_stack2[i=1:num_lines], Bin)
    @constraint(m, constr[i=1:num_lines], 2*line_stack2[i] <= sum(team_lines[k,i]*skaters_lineup[k], k=1:num_skaters))
    @constraint(m, sum(line_stack2[i], i=1:num_lines) >= 2)


    # Overlap Constraint
    @constraint(m, constr[i=1:size(lineups)[2]], sum(lineups[j,i]*skaters_lineup[j], j=1:num_skaters) + sum(lineups[num_skaters+j,i]*goalies_lineup[j], j=1:num_goalies) <= num_overlap)


    # Objective
    @objective(m, Max, sum(skaters[i,:Projection]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Projection]*goalies_lineup[i], i=1:num_goalies) )


    # Solve the integer programming problem
    println("Solving Problem...")
    @printf("\n")
    status = solve(m);


    # Puts the output of one lineup into a format that will be used later
    if status==:Optimal
        skaters_lineup_copy = Array{Int}(undef, 0)
        for i=1:num_skaters
            if getValue(skaters_lineup[i]) >= 0.9 && getValue(skaters_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        for i=1:num_goalies
            if getValue(goalies_lineup[i]) >= 0.9 && getValue(goalies_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        return(skaters_lineup_copy)
    end
end



#=
formulation is the type of formulation that you would like to use. Feel free to customize the formulations. In our paper we considered
the Type 4 formulation in great detail, but we have included the code for all of the formulations dicussed in the paper here. For instance,
if you would like to create lineups without stacking, change one_lineup_Type_4 below to one_lineup_no_stacking
=#
formulation = one_lineup_Type_4








function create_lineups(num_lineups, num_overlap, path_skaters, path_goalies, formulation, path_to_output)
    #=
    num_lineups is an integer that is the number of lineups
    num_overlap is an integer that gives the overlap between each lineup
    path_skaters is a string that gives the path to the skaters csv file
    path_goalies is a string that gives the path to the goalies csv file
    formulation is the type of formulation you would like to use (for instance one_lineup_Type_1, one_lineup_Type_2, etc.)
    path_to_output is a string where the final csv file with your lineups will be
    =#


    # Load information for skaters table
    skaters = DataFrame(CSV.File(HTTP.get(path_skaters).body, normalizenames=true, delim=","))

    # Load information for goalies table
    goalies = DataFrame(CSV.File(HTTP.get(path_goalies).body,normalizenames=true,  delim=","))

    # Number of skaters
    num_skaters = size(skaters)[1]

    # Number of goalies
    num_goalies = size(goalies)[1]

    # wingers stores the information on which players are wings
    wingers = Array{Int}(undef, 0)

    # centers stores the information on which players are centers
    centers = Array{Int}(undef, 0)

    # defenders stores the information on which players are defenders
    defenders = Array{Int}(undef, 0)

    #=
    Process the position information in the skaters file to populate the wingers,
    centers, and defenders with the corresponding correct information
    =#
    for i =1:num_skaters
        if skaters[i,:Position] == "LW" || skaters[i,:Position] == "RW" || skaters[i,:Position] == "W"
            wingers=vcat(wingers,fill(1,1))
            centers=vcat(centers,fill(0,1))
            defenders=vcat(defenders,fill(0,1))
        elseif skaters[i,:Position] == "C"
            wingers=vcat(wingers,fill(0,1))
            centers=vcat(centers,fill(1,1))
            defenders=vcat(defenders,fill(0,1))
        elseif skaters[i,:Position] == "D" || skaters[i,:Position] == "LD" || skaters[i,:Position] == "RD"
            wingers=vcat(wingers,fill(0,1))
            centers=vcat(centers,fill(0,1))
            defenders=vcat(defenders,fill(1,1))
        else
            wingers=vcat(wingers,fill(0,1))
            centers=vcat(centers,fill(0,1))
            defenders=vcat(defenders,fill(1,1))
        end
    end


    # A forward is either a center or a winger
    forwards = centers+wingers



    # Create team indicators from the information in the skaters file
    teams = unique(skaters[!,:Team])

    # Total number of teams
    num_teams = size(teams)[1]

    # player_info stores information on which team each player is on
    player_info = zeros(Int, size(teams)[1])

    # Populate player_info with the corresponding information
    for j=1:size(teams)[1]
        if skaters[1, :Team] == teams[j]
            player_info[j] =1
        end
    end
    skaters_teams = player_info'


    for i=2:num_skaters
        player_info = zeros(Int, size(teams)[1])
        for j=1:size(teams)[1]
            if skaters[1, :Team] == teams[j]
                player_info[j] =1
            end
        end
        skaters_teams = vcat(skaters_teams, player_info')
    end



    # Create goalie identifiers so you know who they are playing
    opponents = goalies[!,:Opponent]
    goalie_teams = goalies[!,:Team]
    goalie_opponents=[]
    for num = 1:size(teams)[1]
        if opponents[1] == teams[num]
            goalie_opponents = skaters_teams[:, num]
        end
    end
    for num = 2:size(opponents)[1]
        for num_2 = 1:size(teams)[1]
            if opponents[num] == teams[num_2]
                goalie_opponents = hcat(goalie_opponents, skaters_teams[:,num_2])
            end
        end
    end




    # Create line indicators so you know which players are on which lines
    L1_info = zeros(Int, num_skaters)
    L2_info = zeros(Int, num_skaters)
    L3_info = zeros(Int, num_skaters)
    L4_info = zeros(Int, num_skaters)
    for num=1:size(skaters)[1]
        if skaters.:Team[num] == teams[1]
            if skaters.:Line[num] == 1
                L1_info[num] = 1
            elseif skaters.:Line[num] == 2
                L2_info[num] = 1
            elseif skaters.:Line[num] == 3
                L3_info[num] = 1
            elseif skaters.:Line[num] == 4
                L4_info[num] = 1
            end
        end
    end
    team_lines = hcat(L1_info, L2_info, L3_info, L4_info)


    for num2 = 2:size(teams)[1]
        L1_info = zeros(Int, num_skaters)
        L2_info = zeros(Int, num_skaters)
        L3_info = zeros(Int, num_skaters)
        L4_info = zeros(Int, num_skaters)
        for num=1:size(skaters)[1]
            if skaters.:Team[num] == teams[num2]
                if skaters.:Line[num] == 1
                    L1_info[num] = 1
                elseif skaters.:Line[num] == 2
                    L2_info[num] = 1
                elseif skaters.:Line[num] == 3
                    L3_info[num] = 1
                elseif skaters.:Line[num] == 4
                    L4_info[num] = 1
                end
            end
        end
        team_lines = hcat(team_lines, L1_info, L2_info, L3_info, L4_info)
    end
    num_lines = size(team_lines)[2]


    # Create power play indicators
    PP_info = zeros(Int, num_skaters)
    for num=1:size(skaters)[1]
        if skaters.:Team[num]==teams[1]
            if skaters.:Power_Play[num] == 1
                PP_info[num] = 1
            end
        end
    end

    P1_info = PP_info

    for num2=2:size(teams)[1]
        PP_info = zeros(Int, num_skaters)
        for num=1:size(skaters)[1]
            if skaters.:Team[num] == teams[num2]
                if skaters.:Power_Play[num] == 1
                    PP_info[num]=1
                end
            end
        end
        P1_info = hcat(P1_info, PP_info)
    end


    # Lineups using formulation as the stacking type
    the_lineup= formulation(skaters, goalies, hcat(zeros(Int, num_skaters + num_goalies), zeros(Int, num_skaters + num_goalies)), num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    the_lineup2 = formulation(skaters, goalies, hcat(the_lineup, zeros(Int, num_skaters + num_goalies)), num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    tracer = hcat(the_lineup, the_lineup2)
    for i=1:(num_lineups-2)
        try
            thelineup=formulation(skaters, goalies, tracer, num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
            tracer = hcat(tracer,thelineup)
        catch
            break
        end
    end


    # Create the output csv file
    lineup2 = ""
    for j = 1:size(tracer)[2]
        lineup = ["" "" "" "" "" "" "" "" ""]
        for i =1:num_skaters
            if tracer[i,j] == 1
                if centers[i]==1
                    if lineup[1]==""
                        lineup[1] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[2]==""
                        lineup[2] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[9] ==""
                        lineup[9] = string(skaters[i,1], " ", skaters[i,2])
                    end
                elseif wingers[i] == 1
                    if lineup[3] == ""
                        lineup[3] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[4] == ""
                        lineup[4] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[5] == ""
                        lineup[5] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[9] == ""
                        lineup[9] = string(skaters[i,1], " ", skaters[i,2])
                    end
                elseif defenders[i]==1
                    if lineup[6] == ""
                        lineup[6] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[7] ==""
                        lineup[7] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[9] == ""
                        lineup[9] = string(skaters[i,1], " ", skaters[i,2])
                    end
                end
            end
        end
        for i =1:num_goalies
            if tracer[num_skaters+i,j] == 1
                lineup[8] = string(goalies[i,1], " ", goalies[i,2])
            end
        end
        for name in lineup
            lineup2 = string(lineup2, name, ",")
        end
        lineup2 = chop(lineup2)
        lineup2 = string(lineup2, """

        """)
    end
    outfile = open(path_to_output, "w")
    write(outfile, lineup2)
    close(outfile)
end




# Running the code
create_lineups(num_lineups, num_overlap, path_skaters, path_goalies, formulation, path_to_output)

Solving Problem...



LoadError: ignored

In [ ]:
skaters = DataFrame(CSV.File(HTTP.get(path_skaters).body, normalizenames=true, delim=","))
print(skaters)

180×9 DataFrame
 Row │ First_Name  Last_Name  Salary  Position  Team     Opponent  Line     Power_Play  Projection 
     │ String7     String7    Int64   String1   String1  String1   String1  String1     Float64    
─────┼─────────────────────────────────────────────────────────────────────────────────────────────
   1 │ John        Doe1         4300  C         A        B         1        1                  7.9
   2 │ John        Doe2         4200  W         A        B         1        1                  6.4
   3 │ John        Doe3         4200  W         A        B         1        1                  4.0
   4 │ John        Doe4         4700  C         A        B         2        1                  4.3
   5 │ John        Doe5         4700  W         A        B         2        2                  5.4
   6 │ John        Doe6         4500  W         A        B         2        2                  3.0
   7 │ John        Doe7         4900  C         A        B         3        2             

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />